<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#The-OpenNotify-API" data-toc-modified-id="The-OpenNotify-API-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>The OpenNotify API</a></span></li><li><span><a href="#HOMEWORK" data-toc-modified-id="HOMEWORK-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>HOMEWORK</a></span></li></ul></div>

![alt text](https://i.imgur.com/1WaY7aA.png)



---



---



# Lab 2.2.1 
# *Querying the International Space Station*

## The OpenNotify API

The OpenNotify API exposes a few attributes of the International Space Station (ISS) via a simple, authentication-free interface. The simplicity of this API precludes any need for a dedicated Python library. However, as with many APIs, it accepts requests according to HTTP standards and returns responses in JSON format, so the Python libraries request and json will make managing the I/O simpler still.

In [1]:
import requests
import json
from datetime import datetime, date, time

This request fetches the latest position of the international space station:

In [2]:
response = requests.get("http://api.open-notify.org/iss-now.json")

Print the status code of the response:

In [3]:
print(response)

<Response [200]>


We can also request the current position of the ISS and the next few times at which it will be over a certain location. The latitude and longitude of Sydney are (-33.87, 151.21). Create a dict named `parameters` whose elements are key:value pairs named `lat` and `lon`, containing those figures:

In [4]:
parameters = {'lat': -33.87, 
              'lon': 151.21}

The API request we want starts like the last one, but the last part of the endpoint is "iss-pass.json", and it takes a second argument -- namely, the `parameters` value created above. Compose and execute the API request:

In [5]:
response = requests.get("http://api.open-notify.org/iss-pass.json", parameters)

Print the response header:

In [6]:
print(response)

<Response [200]>


Print the content of the response (the data that the server returned):

In [7]:
print(response.content)

b'{\n  "message": "success", \n  "request": {\n    "altitude": 100, \n    "datetime": 1553673359, \n    "latitude": -33.87, \n    "longitude": 151.21, \n    "passes": 5\n  }, \n  "response": [\n    {\n      "duration": 111, \n      "risetime": 1553704928\n    }, \n    {\n      "duration": 637, \n      "risetime": 1553710415\n    }, \n    {\n      "duration": 599, \n      "risetime": 1553716236\n    }, \n    {\n      "duration": 401, \n      "risetime": 1553722199\n    }, \n    {\n      "duration": 357, \n      "risetime": 1553728122\n    }\n  ]\n}\n'


Note that this is a Python byte string:

In [8]:
print(response.text)

{
  "message": "success", 
  "request": {
    "altitude": 100, 
    "datetime": 1553673359, 
    "latitude": -33.87, 
    "longitude": 151.21, 
    "passes": 5
  }, 
  "response": [
    {
      "duration": 111, 
      "risetime": 1553704928
    }, 
    {
      "duration": 637, 
      "risetime": 1553710415
    }, 
    {
      "duration": 599, 
      "risetime": 1553716236
    }, 
    {
      "duration": 401, 
      "risetime": 1553722199
    }, 
    {
      "duration": 357, 
      "risetime": 1553728122
    }
  ]
}



Print just the "content-type" value from the header:

In [9]:
print(type(response.content))

<class 'bytes'>


JSON was designed to be easy for computers to read, not for people. The `requests` library can decode the JSON byte string:

In [10]:
response.json()

{'message': 'success',
 'request': {'altitude': 100,
  'datetime': 1553673359,
  'latitude': -33.87,
  'longitude': 151.21,
  'passes': 5},
 'response': [{'duration': 111, 'risetime': 1553704928},
  {'duration': 637, 'risetime': 1553710415},
  {'duration': 599, 'risetime': 1553716236},
  {'duration': 401, 'risetime': 1553722199},
  {'duration': 357, 'risetime': 1553728122}]}

What kind of object did this give us?

In [11]:
type(response.json())

dict

Python dicts are easier to work with, but the data we want is still buried in that data structure, so we have to dig it out. First, extract the `response` value to a separate dict:

In [12]:
response_json = response.json()
iss_location = response_json['response']

iss_location

[{'duration': 111, 'risetime': 1553704928},
 {'duration': 637, 'risetime': 1553710415},
 {'duration': 599, 'risetime': 1553716236},
 {'duration': 401, 'risetime': 1553722199},
 {'duration': 357, 'risetime': 1553728122}]

Now extract the `risetime` strings into an array called `srisetimes`:

In [13]:
srisetimes = [d['risetime'] for d in iss_location]

srisetimes

[1553704928, 1553710415, 1553716236, 1553722199, 1553728122]

```
datetime.fromtimestamp(str)
```

Convert these to an array of Python `datetime` values called `risetimes`:

In [14]:
risetimes = [datetime.fromtimestamp(t) for t in srisetimes]

risetimes

[datetime.datetime(2019, 3, 28, 3, 42, 8),
 datetime.datetime(2019, 3, 28, 5, 13, 35),
 datetime.datetime(2019, 3, 28, 6, 50, 36),
 datetime.datetime(2019, 3, 28, 8, 29, 59),
 datetime.datetime(2019, 3, 28, 10, 8, 42)]

Finally, print these in a format that people understand:



```
str.strftime('%d/%m/%y %I:%M')

27/03/19 10:59
27/03/19 12:35
27/03/19 02:11
28/03/19 03:42
28/03/19 05:13
```



In [15]:
for t in risetimes:
    print(t.strftime('%d/%m/%y %I:%M'))

28/03/19 03:42
28/03/19 05:13
28/03/19 06:50
28/03/19 08:29
28/03/19 10:08


Here is an endpoint that tells us who is onboard:

In [16]:
astronauts = requests.get("http://api.open-notify.org/astros.json")

Referring to the methods used above, extract the number of astronauts and their names:

In [17]:
astro_json = astronauts.json()

astro_names = [d['name'] for d in astro_json['people']]

astro_number = astro_json['number']

In [18]:
astro_names

['Oleg Kononenko',
 'David Saint-Jacques',
 'Anne McClain',
 'Alexey Ovchinin',
 'Nick Hague',
 'Christina Koch']

In [19]:
astro_number

6

In [21]:
type(response.status_code)

int

## HOMEWORK


1. Write a simple handler for the response status code (refer to lab resources slide for HTTP response codes). As this Jupyter Notebook is an interactive device, the handler does not need to manage subsequent code execution (i.e. by branching or aborting execution), although it should return something that could be used to do so if deployed in a Python program.

In [24]:
#ANSWER:
def handleResponse(response, verbose = False):
    '''
    Returns Boolean Value, Status Code, 
    
    '''
  # if Status Code is 200 return false, and status code
  # Otherwise Return True and Status Code
    
    if response.status_code == 200:
        return False, response.status_code
    else:
        return True, response.status_code

2. Test your response handler on some correct and incorrect API calls.

In [26]:
response = requests.get("http://api.open-notify.org/astros.json")
if handleResponse(response)[0]:
    print('API call failed. Resolve issue before continuing!')
else:
    print("API call successful")
    
    
response = requests.get("http://api.open-notify.org/iss-pass.json")
handleResponse(response, True)[0]

API call successful


True

>

>

>



---



---


> > > > > > > > >© 2019 Data Science Institute of Australia


---



---

